In [ ]:
import sys, time, os, asyncio, glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from astropy.time import Time, TimeDelta
from lsst_efd_client import EfdClient
from lsst.summit.utils.utils import dayObsIntToString
from lsst.summit.utils.efdUtils import calcNextDay
%matplotlib inline

In [ ]:
client = EfdClient('usdf_efd')

In [ ]:
start = Time("2024-03-12T03:10:00", scale='utc')
end = Time("2024-03-12T03:13:00", scale='utc')


In [ ]:
dome = await client.select_time_series('lsst.sal.ATDome.position', \
                                                    ['*'],  start, end)

In [ ]:
fig, axs = plt.subplots(2,1,figsize=(5,5))
plt.subplots_adjust(hspace=0.6)
axs[0].set_title("Azimuth Position (deg)")
axs[0].set_ylabel("Degrees")
dome['azimuthPosition'].plot(ax=axs[0])
axs[1].set_title("Azimuth Encoder (no units)")
axs[1].set_ylabel("Encoder Value")
dome['azimuthEncoderPosition'].plot(ax=axs[1])
plt.savefig("/home/c/cslage/u/AuxTel/dome/Azimuth_Overshoots_12Mar24.png")

In [ ]:
start = Time("2024-03-12T00:00:00", scale='utc')
end = Time("2024-03-12T04:00:00", scale='utc')

dome = await client.select_time_series('lsst.sal.ATDome.position', \
                                                    ['*'],  start, end)
move = await client.select_time_series('lsst.sal.ATDome.command_moveAzimuth', \
                                                    ['*'],  start, end)
inPos = await client.select_time_series('lsst.sal.ATDome.logevent_azimuthInPosition', \
                                                    ['*'],  start, end)
inPosTrue = inPos[inPos['inPosition'] == True]
len(inPosTrue)
inPosFalse = inPos[inPos['inPosition'] == False]


In [ ]:
fig, axs = plt.subplots(2,1,figsize=(10,5))
plt.subplots_adjust(hspace=0.6)
axs[0].set_title("Azimuth Position (deg)")
axs[0].set_ylabel("Degrees")
dome['azimuthPosition'].plot(ax=axs[0])
for pos in inPosTrue.index:
    axs[0].axvline(pos, color='green', ls='--')
    axs[1].axvline(pos, color='green', ls='--')
#for mov in move.index:
#    axs[0].axvline(mov, color='red', ls='--')

axs[1].set_title("Azimuth Encoder (no units)")
axs[1].set_ylabel("Encoder Value")
dome['azimuthEncoderPosition'].plot(ax=axs[1])
plt.savefig("/home/c/cslage/u/AuxTel/dome/Azimuth_InPos_12Mar24.png")

In [ ]:
start = Time("2024-03-12T00:00:00", scale='utc')
end = Time("2024-03-12T04:00:00", scale='utc')

dome = await client.select_time_series('lsst.sal.ATDome.position', \
                                                    ['*'],  start, end)
inPos = await client.select_time_series('lsst.sal.ATDome.logevent_azimuthInPosition', \
                                                    ['*'],  start, end)
inPosTrue = inPos[inPos['inPosition'] == True]
len(inPosTrue)
inPosFalse = inPos[inPos['inPosition'] == False]
fail = await client.select_time_series('lsst.sal.ATDome.logevent_logMessage', \
                                                    ['*'],  start, end)
fails = []
for i in range(len(fail)):
    if fail['message'].iloc[i].split(' ')[0] == 'Dome':
        print(fail['message'].iloc[i])
        fails.append(fail.index[i])


In [ ]:
fig, axs = plt.subplots(2,1,figsize=(10,5))
plt.subplots_adjust(hspace=0.6)
axs[0].set_title("Azimuth Position (deg)")
axs[0].set_ylabel("Degrees")
dome['azimuthPosition'].plot(ax=axs[0])
for pos in inPosTrue.index:
    axs[0].axvline(pos, color='green', ls='--')
    axs[1].axvline(pos, color='green', ls='--')
for thisFail in fails:
    axs[0].axvline(thisFail, color='red', ls='--')
    axs[1].axvline(thisFail, color='red', ls='--')
axs[1].set_title("Azimuth Encoder (no units)")
axs[1].set_ylabel("Encoder Value")
dome['azimuthEncoderPosition'].plot(ax=axs[1])
plt.savefig("/home/c/cslage/u/AuxTel/dome/Azimuth_Fails_12Mar24.png")

In [ ]:
start = Time("2024-03-12T00:07:00", scale='utc')
end = Time("2024-03-12T00:10:00", scale='utc')

dome = await client.select_time_series('lsst.sal.ATDome.position', \
                                                    ['*'],  start, end)
inPos = await client.select_time_series('lsst.sal.ATDome.logevent_azimuthInPosition', \
                                                    ['*'],  start, end)
inPosTrue = inPos[inPos['inPosition'] == True]
len(inPosTrue)
inPosFalse = inPos[inPos['inPosition'] == False]
fail = await client.select_time_series('lsst.sal.ATDome.logevent_logMessage', \
                                                    ['*'],  start, end)
fails = []
for i in range(len(fail)):
    if fail['message'].iloc[i].split(' ')[0] == 'Dome':
        print(fail['message'].iloc[i])
        fails.append(fail.index[i])


In [ ]:
fig, axs = plt.subplots(2,1,figsize=(10,5))
plt.subplots_adjust(hspace=0.6)
axs[0].set_title("Azimuth Position (deg)")
axs[0].set_ylabel("Degrees")
dome['azimuthPosition'].plot(ax=axs[0])
for pos in inPosTrue.index:
    axs[0].axvline(pos, color='green', ls='--', label='InPosition')
    axs[1].axvline(pos, color='green', ls='--')
for thisFail in fails:
    axs[0].axvline(thisFail, color='red', ls='--', label='Not moving, resending')
    axs[1].axvline(thisFail, color='red', ls='--')
axs[1].set_title("Azimuth Encoder (no units)")
axs[1].set_ylabel("Encoder Value")
dome['azimuthEncoderPosition'].plot(ax=axs[1])
axs[0].legend()
plt.savefig("/home/c/cslage/u/AuxTel/dome/Azimuth_Fail_BlowUp_12Mar24.png")

In [ ]:
start = Time("2024-03-12T01:41:00", scale='utc')
end = Time("2024-03-12T01:41:30", scale='utc')

dome = await client.select_time_series('lsst.sal.ATDome.position', \
                                                    ['*'],  start, end)
move = await client.select_time_series('lsst.sal.ATDome.command_moveAzimuth', \
                                                    ['*'],  start, end)
inPos = await client.select_time_series('lsst.sal.ATDome.logevent_azimuthInPosition', \
                                                    ['*'],  start, end)
inPosTrue = inPos[inPos['inPosition'] == True]

inPosFalse = inPos[inPos['inPosition'] == False]
len(move)

In [ ]:
fig, axs = plt.subplots(2,1,figsize=(10,5))
plt.subplots_adjust(hspace=0.6)
axs[0].set_title("Azimuth Position (deg)")
axs[0].set_ylabel("Degrees")
dome['azimuthPosition'].plot(ax=axs[0])
for pos in inPosTrue.index:
    axs[0].axvline(pos, color='green', ls='--')
    axs[1].axvline(pos, color='green', ls='--')
for mov in move.index:
    axs[0].axvline(mov, color='red', ls='--')

axs[1].set_title("Azimuth Encoder (no units)")
axs[1].set_ylabel("Encoder Value")
dome['azimuthEncoderPosition'].plot(ax=axs[1])
plt.savefig("/home/c/cslage/u/AuxTel/dome/Azimuth_Move_BlowUp_12Mar24.png")

In [ ]:
start = Time("2024-03-12T00:00:00", scale='utc')
end = Time("2024-03-12T04:00:00", scale='utc')

dome = await client.select_time_series('lsst.sal.ATDome.position', \
                                                    ['*'],  start, end)
move = await client.select_time_series('lsst.sal.ATDome.command_moveAzimuth', \
                                                    ['*'],  start, end)
inPos = await client.select_time_series('lsst.sal.ATDome.logevent_azimuthInPosition', \
                                                    ['*'],  start, end)
inPosTrue = inPos[inPos['inPosition'] == True]
len(inPosTrue)
inPosFalse = inPos[inPos['inPosition'] == False]


In [ ]:
fig, axs = plt.subplots(2,1,figsize=(10,5))
plt.subplots_adjust(hspace=0.6)
axs[0].set_title("Azimuth Position (deg)")
axs[0].set_ylabel("Degrees")
dome['azimuthPosition'].plot(ax=axs[0])
for pos in inPosTrue.index:
    axs[0].axvline(pos, color='green', ls='--')
    axs[1].axvline(pos, color='green', ls='--')
for mov in move.index:
    axs[0].axvline(mov, color='red', ls='--')

axs[1].set_title("Azimuth Encoder (no units)")
axs[1].set_ylabel("Encoder Value")
dome['azimuthEncoderPosition'].plot(ax=axs[1])
plt.savefig("/home/c/cslage/u/AuxTel/dome/Azimuth_Move_12Mar24.png")

In [ ]:
encoder_rollover = 34698002608 # I think this is encoder full scale
encoder_counts_per_rotation = 4018143232
current_encoder = 21811220507

In [ ]:
# How many rotations before we roll over?
encoder_rollover / encoder_counts_per_rotation

In [ ]:
# How far are we from rollover?
(encoder_rollover - current_encoder) / encoder_counts_per_rotation